In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv ('data/titanic_train_master.csv')
test = pd.read_csv ('data/titanic_test_master.csv')

In [4]:
# combined all data for feature creation
# this removes potential for dummies in one set and not the other
train_end_row = train.shape[0]
test_end_row = test.shape[0]
combined = pd.concat ([train, test])

In [5]:
# lower case all column names
combined.columns = map (str.lower, combined.columns)

In [6]:
# dummies function to create binary columns for groups
def create_dummies (df, column_names):
    for col in column_names:
        dummies = pd.get_dummies (df[col], prefix=col)
        df = pd.concat([df, dummies], axis=1)
    return df

In [7]:
# Try combining Pclass + Sex
combined['psex'] = combined['pclass'] + combined['sex'].map({'male':0, 'female':1})

In [8]:
# PCLASS
combined = create_dummies (combined, ['pclass'])

In [9]:
# extract titles from name
titles = {
    "Mr" :         "mr",
    "Mme":         "mme",
    "Ms":          "mme",
    "Mrs" :        "mrs",
    "Master" :     "master",
    "Mlle":        "miss",
    "Miss" :       "miss",
    "Capt":        "officer",
    "Col":         "officer",
    "Major":       "officer",
    "Dr":          "officer",
    "Rev":         "officer",
    "Jonkheer":    "royalty_male",
    "Don":         "royalty_male",
    "Sir" :        "royalty_male",
    "Countess":    "royalty_female",
    "Dona":        "royalty_female",
    "Lady" :       "royalty_female"
}
extracted_titles = combined['name'].str.extract (' ([A-Za-z]+)\.', expand=False)
combined['title'] = extracted_titles.map(titles)

# create dummies for titles
combined = create_dummies (combined, ['title'])

In [10]:
# SEX
# train['sex'] = train['sex'].map({'male':0, 'female':1}) 
# test['sex'] = test['sex'].map({'male':0, 'female':1})
combined = create_dummies (combined, ['sex'])

In [11]:
# AGE
# if title_master is true and age is null set age to 1. Then group ages. 
# Trying to save some rows
def set_age (row):
    if row['title_master'] & pd.isnull(row['age']):
        return 1
    else:
        return row['age']

combined['age'] = combined.apply (lambda row: set_age(row), axis=1)

# create age groups.
def create_groups_dummies (df, col, cat_name, cut_points, label_names):
    df[col] = df[col].fillna(-0.5)
    df[cat_name] = pd.cut (df[col], cut_points, labels=label_names)
    df = create_dummies (df, [cat_name])
    return df

age_cut_points = [-1, 0, 12, 18, 40, 60, 100]
age_group_labels = ['missing', 'child', 'teenager', 'young_adult', 'adult', 'senior']

combined = create_groups_dummies (combined, 'age', 'age_cat', age_cut_points, 
                                    age_group_labels)

In [12]:
# Create a FAMILY feature
# This will capture sibsp and parch so they can be dropped
combined['family'] = combined['sibsp'] + combined['parch'] + 1
combined = create_dummies (combined, ['family'])

In [13]:
# TICKET
# Extract ticket number and then assume all with the same number are traveling together
ticket_split = combined['ticket'].str[::-1].str.split(' ', 1, expand=True)
combined['ticketnum'] = ticket_split[0].str[::-1]
combined['ticketnum'] = combined['ticketnum'].str.replace("LINE", '0')
combined['ticketnum'] = combined['ticketnum'].apply(pd.to_numeric)
# fam_groups = combined['ticketnum'].value_counts()
combined['ticket_group'] = 0
tgroup = {}
groupnum = 0
for i, row in combined.iterrows():
    ticketnum = row['ticketnum']
    if ticketnum in tgroup and (row['title_master'] or row['sex_female']):
        combined.at[i, 'ticket_group'] = tgroup[ticketnum]
    else:
        tgroup[ticketnum] = groupnum
        combined.at[i, 'ticket_group'] = groupnum
        groupnum += 1
combined.drop(['ticketnum'], 1, inplace=True)

In [14]:
# FARE
# Find the one null rec and see if we can fill it
combined.loc[combined['fare'].isnull()]

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,...,family_1,family_2,family_3,family_4,family_5,family_6,family_7,family_8,family_11,ticket_group
152,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,...,1,0,0,0,0,0,0,0,0,878


In [15]:
# the one nan can be filled with the average for the passenger
filler = combined.loc[(combined['pclass']==3) & 
                        (combined['sex']=="male") & 
                        (combined['age']>55) & 
                        (combined['age']<65)]['fare'].dropna().mean()
combined['fare'] = combined['fare'].fillna(filler)
fare_group_labels = ['missing', '0-12', '12-50', '50-100', '100+']
fare_cut_points = [-1, 0, 12, 50, 100, 1000]
# fare_cut_points = [-1, 0, 172, 342, 1000]
combined = create_groups_dummies (combined, 'fare', 'fare_cat', 
                                fare_cut_points, fare_group_labels)

In [16]:
# CABIN
combined['cabin_type'] = combined['cabin'].astype(str).str[0]
combined = create_dummies (combined, ['cabin_type'])

In [17]:
# EMBARKED 
# fill empties with the most frequent
# C = Cherbourg, Q = Queenstown, S = Southampton
most_frequent = combined['embarked'].value_counts().index[0]
combined['embarked'] = combined['embarked'].fillna(most_frequent)
combined = create_dummies (combined, ['embarked'])

In [18]:
# clean up
combined.drop(['pclass', 'name', 'ticket', 'title', 'sex', 'age', 'family', 'sibsp', 'parch', 'embarked', 'cabin', 'fare', 'fare_cat', 'cabin_type', 'age_cat'], 1, inplace=True)
combined.columns = map (str.lower, combined.columns)

In [19]:
# now split back into train and test. Drop survived from test
train_final = combined.iloc[:train_end_row]
test_final = combined.iloc[train_end_row:].drop(['survived'], 1)

In [20]:
# write out final files
train_final.to_csv ('data/titanic_train_wrangled.csv', index=False)
test_final.to_csv ('data/titanic_test_wrangled.csv', index=False)

In [22]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   passengerid           891 non-null    int64  
 1   survived              891 non-null    float64
 2   psex                  891 non-null    int64  
 3   pclass_1              891 non-null    uint8  
 4   pclass_2              891 non-null    uint8  
 5   pclass_3              891 non-null    uint8  
 6   title_master          891 non-null    uint8  
 7   title_miss            891 non-null    uint8  
 8   title_mme             891 non-null    uint8  
 9   title_mr              891 non-null    uint8  
 10  title_mrs             891 non-null    uint8  
 11  title_officer         891 non-null    uint8  
 12  title_royalty_female  891 non-null    uint8  
 13  title_royalty_male    891 non-null    uint8  
 14  sex_female            891 non-null    uint8  
 15  sex_male              8